# Valores faltantes / missing values. Caso: variables numéricas
* Los valores faltantes suceden.
* Veamos tres enfoques para gestionar los valores faltantes.
* Luego se comparará la efectividad de estos enfoques en un conjunto de datos del mundo real.

**Introducción**
* Hay muchas formas en que los datos pueden terminar con valores faltantes. Por ejemplo,
  * Una casa de 2 dormitorios no incluirá el valor del tamaño de un tercer dormitorio.
  * Un encuestado puede optar por no compartir sus ingresos.
* La mayoría de las bibliotecas de aprendizaje automático (incluida scikit-learn) dan un error si intentas construir un modelo usando datos con valores faltantes. Por lo tanto, deberá elegir una de las estrategias siguientes.

### Enfoque 1. **Opcion Simple**. Eliminar columnas con valores faltantes
* La opción más sencilla es eliminar las columnas con valores faltantes.
* A menos que falten la mayoría de los valores en las columnas eliminadas, el modelo pierde acceso a mucha información (¡potencialmente útil!) con este enfoque.

### Enfoque 2. **Imputación**
* La imputación completa los valores faltantes con algún número.
* Por ejemplo, podemos completar el valor medio en cada columna.
* El valor imputado no será exactamente correcto en la mayoría de los casos, pero generalmente conduce a modelos más precisos que los que se obtendrían si eliminara la columna por completo.

### Enfoque 3. **Imputación** con dataframe extendido
* La imputación es el enfoque estándar y normalmente funciona bien.
* Sin embargo, los valores imputados pueden estar sistemáticamente por encima o por debajo de sus valores reales (que no se recopilaron en el conjunto de datos).
* O las filas con valores faltantes pueden ser únicas de alguna otra manera. En ese caso, su modelo haría mejores predicciones al considerar qué valores faltaban originalmente.
* Con este enfoque, imputamos los valores faltantes, como antes. **Y, además, para cada columna a la que le faltan entradas en el conjunto de datos original, agregamos una nueva columna que muestra la ubicación de las entradas imputadas.**
* En algunos casos, esto mejorará significativamente los resultados. En otros casos, no ayuda en absoluto.

## Cargar el dataset

* Estamos trabajando con el **dataset de Melbourne Housing** (https://www.kaggle.com/datasets/dansbecker/melbourne-housing-snapshot). 
* Nuestro modelo utilizará información del dataset para predecir el precio de la vivienda.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Leer el archivo 'train.csv' y asignarlo a X_full
X_full= pd.read_csv('train.csv', index_col='Id')

# Leer el archivo 'test.csv' y asignarlo a X_test_full
# Este archjivo se utiliza para correr el modelo test
X_test_full = pd.read_csv('test.csv', index_col='Id')

In [3]:
X_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

X_full, tiene: 
* 1460 filas y 80 columnas, entre las cuales existen:
    *  37 variables numéricas entre int64(34) y float64(3)
    *  Ademas, 43 variables catgóricas tipo object
* Asimismo, se evidencian valores faltantes

In [4]:
X_test_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   Alley          107 non-null    object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearB

* X_test_full, tiene 79 columnas con (36) datos numericos: float64(11), int64(25) y 43 datos categóricos (tipo=object).
* Tambien existen valores faltantes.

## Cargar datos de entrenamiento, validación y prueba.(Caso: solo variables numéricas)
* Este caso se correrá con sólo varaibles numéricas. 
* Cargamos los conjuntos de entrenamiento y validación en **X_train, X_valid, y_train e y_valid.**
* El conjunto de prueba se carga en **X_test_full**

In [5]:
# 1. Eliminar filas con valores faltantes relacionados a la columna objetivo "SalePrice",
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)

In [6]:
# 2. Definir variable objetivo (y) y variables predictoras (X)
y = X_full.SalePrice
# Separar el objetivo de los predictores
X_full.drop(['SalePrice'], axis=1, inplace=True)

In [8]:
# 2.1 En este caso,por conveniencia, usaremos solo predictores numéricos
# Se eliminan las variables categóricas
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

In [9]:
# 2.2 Vistazo a los dataframe
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   LotFrontage    1201 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   OverallQual    1460 non-null   int64  
 4   OverallCond    1460 non-null   int64  
 5   YearBuilt      1460 non-null   int64  
 6   YearRemodAdd   1460 non-null   int64  
 7   MasVnrArea     1452 non-null   float64
 8   BsmtFinSF1     1460 non-null   int64  
 9   BsmtFinSF2     1460 non-null   int64  
 10  BsmtUnfSF      1460 non-null   int64  
 11  TotalBsmtSF    1460 non-null   int64  
 12  1stFlrSF       1460 non-null   int64  
 13  2ndFlrSF       1460 non-null   int64  
 14  LowQualFinSF   1460 non-null   int64  
 15  GrLivArea      1460 non-null   int64  
 16  BsmtFullBath   1460 non-null   int64  
 17  BsmtHalfBath   1460 non-null   int64  
 18  FullBath

In [10]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   LotFrontage    1232 non-null   float64
 2   LotArea        1459 non-null   int64  
 3   OverallQual    1459 non-null   int64  
 4   OverallCond    1459 non-null   int64  
 5   YearBuilt      1459 non-null   int64  
 6   YearRemodAdd   1459 non-null   int64  
 7   MasVnrArea     1444 non-null   float64
 8   BsmtFinSF1     1458 non-null   float64
 9   BsmtFinSF2     1458 non-null   float64
 10  BsmtUnfSF      1458 non-null   float64
 11  TotalBsmtSF    1458 non-null   float64
 12  1stFlrSF       1459 non-null   int64  
 13  2ndFlrSF       1459 non-null   int64  
 14  LowQualFinSF   1459 non-null   int64  
 15  GrLivArea      1459 non-null   int64  
 16  BsmtFullBath   1457 non-null   float64
 17  BsmtHalfBath   1457 non-null   float64
 18  FullB

* Nuestros dataframe se redujeron a 36 columnas y 1460 filas con variables numéricas.

In [11]:
# 3. Separar el conjunto de validación de los datos de entrenamiento
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

* Como ya sabemos los modelos de Machine Leraning no corren con conjuntos que tengan datos faltantes.
* Veamos a X_train

In [12]:
# Un vistazo a las primeras filas de X_train
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [13]:
X_train.tail()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
764,60,82.0,9430,8,5,1999,1999,673.0,1163,0,...,856,0,128,0,0,180,0,0,7,2009
836,20,60.0,9600,4,7,1950,1995,0.0,442,0,...,436,290,0,0,0,0,0,0,2,2010
1217,90,68.0,8930,6,5,1978,1978,0.0,0,0,...,539,0,0,0,0,0,0,0,4,2010
560,120,NaN,3196,7,5,2003,2004,18.0,0,0,...,420,143,20,0,0,0,0,0,10,2006
685,60,58.0,16770,7,5,1998,1998,30.0,0,0,...,486,0,81,0,0,0,0,0,6,2010


* Se observan valores faltantes (NaN) en las primeras y ultimas filas de X_train

## Información preliminar más detallada
* Tamaño de los conjuntos de datos 
* Valores faltantes en cada uno de ellos.

In [17]:
print(f"X_train tiene {X_train.shape}, filas y columnas")
print(f"y_train tiene {y_train.shape}, filas y columnas")
print(f"X_valid tiene {X_valid.shape}, filas y columnas")
print(f"y_valid tiene {y_valid.shape}, filas y columnas")

X_train tiene (1168, 36), filas y columnas
y_train tiene (1168,), filas y columnas
X_valid tiene (292, 36), filas y columnas
y_valid tiene (292,), filas y columnas


## Valores faltantes en las columnas de X_train

In [43]:
# Identificar en una lista las columnas con valores faltantes en X_train y almacenar en una variable
cols_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]
cols_missing

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [47]:
# Cantidad de valores faltantes en cada columna de datos de entrenamiento
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


## Valores faltantes en el conjunto de prueba: X_test

In [20]:
# Identificar las columnas con valores faltantes en X_test y almacenar en una variable
cols_missing_test = [col for col in X_test.columns
                     if X_test[col].isnull().any()]
cols_missing_test

['LotFrontage',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea']

In [22]:
# Cantidad de valores faltantes en cada columna de datos test
missing_val_count_by_column_test = (X_test.isnull().sum())
print(missing_val_count_by_column_test[missing_val_count_by_column_test > 0])

LotFrontage     227
MasVnrArea       15
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
GarageYrBlt      78
GarageCars        1
GarageArea        1
dtype: int64


* En X_test, existen 8 variables adicionales a las 3 con valores faltantes en X_train.

In [55]:
print(f"X_train tiene {X_train.shape[0]} filas")
print(f"Existen {len(cols_missing)} columnas en los datos de entranamiento con valores faltantes")
print(f"Hay {(missing_val_count_by_column[missing_val_count_by_column > 0].sum())} entradas con valores faltantes")
print(f"El porcentaje de datos faltantes en X_train es de un {(missing_val_count_by_column[missing_val_count_by_column > 0].sum())/X_train.shape[0]*100} %")

X_train tiene 1168 filas
Existen 3 columnas en los datos de entranamiento con valores faltantes
Hay 276 entradas con valores faltantes
El porcentaje de datos faltantes en X_train es de 23.63013698630137 %


In [56]:
print(f"X_test tiene {X_test.shape[0]} filas")
print(f"Existen {len(cols_missing_test)} columnas en los datos de prueba con valores faltantes")
print(f"Hay {(missing_val_count_by_column_test[missing_val_count_by_column_test > 0].sum())} entradas con valores faltantes")
print(f"El porcentaje de datos faltantes en X_test es de un {(missing_val_count_by_column_test[missing_val_count_by_column_test > 0].sum())/X_test.shape[0]*100} %")

X_test tiene 1459 filas
Existen 11 columnas en los datos de prueba con valores faltantes
Hay 330 entradas con valores faltantes
El porcentaje de datos faltantes en X_test es de un 22.618231665524334 %


## Funcion para comparar los diferentes enfoques para tratar los valores faltantes.
* Para comparar diferentes enfoques para tratar los valores faltantes, se usará una función de utilidad
* Esta función informa el error absoluto medio (MAE) de un modelo de bosque aleatorio.

In [57]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)# Fue nuestro mejor modelo antes
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## Enfoque 1: Eliminar columnas con valores faltantes
* En este paso, se preprocesará los datos en X_train y X_valid para eliminar columnas con valores faltantes.
* Se establecen los DataFrames preprocesados en **reduce_X_train** y **reduce_X_valid**, respectivamente.
* Recordemos que **cols_missing** contien las columnas con valores faltantes en X_train.

In [60]:
# 1. Eliminar las columnas con valores faltantos tanto en los datos de entrenamiento como en los de validación.
# Asignar los nombres: reducido_X_train y reducido_X_valid, respectivamente.
reduce_X_train = X_train.drop(cols_missing, axis=1)
reduce_X_valid = X_valid.drop(cols_missing, axis=1)

Verificamos el numéro de filas y columnas de cada conjunto de datos

In [61]:
print(f"reduce_X_train tiene {reduce_X_train.shape}, filas y columnas")
print(f"y_train tiene {y_train.shape}, filas y columnas")
print(f"reduce_X_valid tiene {reduce_X_valid.shape}, filas y columnas")
print(f"y_valid tiene {y_valid.shape}, filas y columnas")

reduce_X_train tiene (1168, 33), filas y columnas
y_train tiene (1168,), filas y columnas
reduce_X_valid tiene (292, 33), filas y columnas
y_valid tiene (292,), filas y columnas


* Se eliminaron las tres columnas con valores faltantes, de 36 pasamos a 33 columnas. 

In [62]:
# 2. Obtener el MAE para este enfoque.
print("MAE (Eliminar columnas con valores faltantes):")
print(score_dataset(reduce_X_train, reduce_X_valid, y_train, y_valid))

MAE (Eliminar columnas con valores faltantes):
17837.82570776256


## Enfoque 2 Imputación Simple

* Imputar los valores faltantes con el valor medio, la mediana y valor mas frecuente, a lo largo de cada columna.
* Establecer los DataFrames preprocesados en **imputed_X_train** e **imputed_X_valid**.
* Asegurar de que los nombres de las columnas coincidan con los de X_train y X_valid.

### 2.1 Imputación Simple (strategy='mean')

In [63]:
# 1. Imputación Simple (mean)
from sklearn.impute import SimpleImputer
mi_imputacion = SimpleImputer(strategy='mean')

imputed_mean_X_train = pd.DataFrame(mi_imputacion.fit_transform(X_train))
imputed_mean_X_valid = pd.DataFrame(mi_imputacion.transform(X_valid))

In [64]:
# 2. Devolver nombre a columnas eliminadas por la imputación.

imputed_mean_X_train.columns = X_train.columns
imputed_mean_X_valid.columns = X_valid.columns

In [65]:
# 3. Revision del numero de filas y columnas de cada conjunto de datos
print(f"imputed_mean_X_train tiene {imputed_mean_X_train.shape}, filas y columnas")
print(f"y_train tiene {y_train.shape}, filas y columnas")
print(f"imputed_mean_X_valid tiene {imputed_mean_X_valid.shape}, filas y columnas")
print(f"y_valid tiene {y_valid.shape}, filas y columnas")

imputed_mean_X_train tiene (1168, 36), filas y columnas
y_train tiene (1168,), filas y columnas
imputed_mean_X_valid tiene (292, 36), filas y columnas
y_valid tiene (292,), filas y columnas


In [67]:
# 4. Obtener el MAE para este enfoque.
print("MAE (Imputación Simple: media):")
print(score_dataset(imputed_mean_X_train, imputed_mean_X_valid, y_train, y_valid))

MAE (Imputación Simple: media):
18062.894611872147


* Dado que hay tan pocos valores faltantes en el conjunto de datos, esperaríamos que la **imputación funcionara mejor** que **eliminar columnas** por completo.
* Sin embargo, **vemos que eliminar columnas funciona ligeramente mejor**.
* Si bien esto probablemente pueda atribuirse en parte al ruido en el conjunto de datos, otra posible explicación **es que el método de imputación no coincide muy bien con este conjunto de datos**.
* Es decir, tal vez en lugar de completar el valor medio, tenga más sentido establecer cada valor faltante en un valor de 0, completar el valor encontrado con más frecuencia o utilizar algún otro método.
* Por ejemplo, consideremos la columna **GarageYrBlt** (que indica el año en que se construyó el garaje).
* Es probable que, en algunos casos, un valor faltante indique una casa que no tiene garaje.
  * ¿Tiene más sentido completar el **valor mediano** en cada columna en este caso?
  * ¿O podríamos obtener mejores resultados completando el **valor mínimo** en cada columna?
* No está del todo claro qué es lo mejor en este caso, pero tal vez podamos descartar algunas opciones de inmediato; por ejemplo, **establecer los valores faltantes en esta columna en 0** probablemente produzca resultados horribles.

### 2.2 Imputación Simple (strategy='median')

In [77]:
#2.2.1  Imputación median
from sklearn.impute import SimpleImputer
mi_imputacion = SimpleImputer(strategy='median')

imputed_median_X_train = pd.DataFrame(mi_imputacion.fit_transform(X_train))
imputed_median_X_valid = pd.DataFrame(mi_imputacion.transform(X_valid))

# 2.2.2. Devolver nombre a columnas eliminadas por la imputación.

imputed_median_X_train.columns = X_train.columns
imputed_median_X_valid.columns = X_valid.columns

# 2.2.3 Obtener el MAE para este enfoque.
print("MAE (Imputation Simple: mediana):")
print(score_dataset(imputed_median_X_train, imputed_median_X_valid, y_train, y_valid))

MAE (Imputation Simple: mediana):
17791.59899543379


### 2.3 Imputación Simple (strategy='most_frequent')

In [71]:
# 2.3.1. Imputación most_frequent
from sklearn.impute import SimpleImputer
mi_imputacion = SimpleImputer(strategy='most_frequent')

imputed_mf_X_train = pd.DataFrame(mi_imputacion.fit_transform(X_train))
imputed_mf_X_valid = pd.DataFrame(mi_imputacion.transform(X_valid))

# 2.3.2. Devolver nombre a columnas eliminadas por la imputación.

imputed_mf_X_train.columns = X_train.columns
imputed_mf_X_valid.columns = X_valid.columns

# 2.3.3. Obtener el MAE para este enfoque.
print("MAE (Imputation Simple: most_frequent):")
print(score_dataset(imputed_mf_X_train, imputed_mf_X_valid, y_train, y_valid))

MAE (Imputation Simple: most_frequent):
17956.065479452056


## Enfoque 3: Imputación con dataframe extendido

In [72]:
# 1. Hacer una copia para evitar cambiar los datos originales, al cambiar
X_train_extended = X_train.copy()
X_valid_extended = X_valid.copy()

# 2. Identificar las columnas con valores faltantes y almacenar en una variable: "cols_with_missing_extended"
cols_with_missing_extended = [col for col in X_train_extended.columns
                     if X_train_extended[col].isnull().any()]

# 2.1 Crear las nuevas columnas para las variables que se imputaran (3 variables)
# Se utiliza la lista definida: "cols_with_missing_extended"


for col in cols_with_missing_extended:
    X_train_extended[col + '_was_missing'] = X_train_extended[col].isnull()
    X_valid_extended[col + '_was_missing'] = X_valid_extended[col].isnull()

# 2.2 Revisar X_train_extended
X_train_extended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168 entries, 619 to 685
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   MSSubClass               1168 non-null   int64  
 1   LotFrontage              956 non-null    float64
 2   LotArea                  1168 non-null   int64  
 3   OverallQual              1168 non-null   int64  
 4   OverallCond              1168 non-null   int64  
 5   YearBuilt                1168 non-null   int64  
 6   YearRemodAdd             1168 non-null   int64  
 7   MasVnrArea               1162 non-null   float64
 8   BsmtFinSF1               1168 non-null   int64  
 9   BsmtFinSF2               1168 non-null   int64  
 10  BsmtUnfSF                1168 non-null   int64  
 11  TotalBsmtSF              1168 non-null   int64  
 12  1stFlrSF                 1168 non-null   int64  
 13  2ndFlrSF                 1168 non-null   int64  
 14  LowQualFinSF           

* A cada columna con valores faltantes, Se han agregado una adicional. Las 36, 37 y 38, tipo booleanas.
* Veamos ahora cada caso de imputación con este dataframe extendido.

### 3.1 Imputación dataframe extendido (strategy='mean')

In [73]:
# 3.1.1 Imputation (media)
my_imputer = SimpleImputer()

imputed_mean_X_train_extended = pd.DataFrame(my_imputer.fit_transform(X_train_extended))
imputed_mean_X_valid_extended = pd.DataFrame(my_imputer.transform(X_valid_extended))

# 3.1.2 Devolver el nombre dev las columnas, al imputar se elimanaron los nombres
imputed_mean_X_train_extended.columns = X_train_extended.columns
imputed_mean_X_valid_extended.columns = X_valid_extended.columns

# 3.1.3. Calculo del MAE para el Método
print("MAE from Approach 3 (An Extension to Imputation: mean):")
print(score_dataset(imputed_mean_X_train_extended, imputed_mean_X_valid_extended, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation: mean):
18148.417180365297


### 3.2 Imputación (strategy='median')

In [74]:
# 3.2.1. Imputation (mediana)
my_imputer = SimpleImputer(strategy='median')

imputed_median_X_train_extended = pd.DataFrame(my_imputer.fit_transform(X_train_extended))
imputed_median_X_valid_extended = pd.DataFrame(my_imputer.transform(X_valid_extended))

# 3.2.2 Devolver el nombre dev las columnas, al imputar se elimanaron los nombres
imputed_median_X_train_extended.columns = X_train_extended.columns
imputed_median_X_valid_extended.columns = X_valid_extended.columns

# 3.2.3 Calculo del MAE para el Método
print("MAE from Approach 3 (An Extension to Imputation: median):")
print(score_dataset(imputed_median_X_train_extended, imputed_median_X_valid_extended, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation: median):
18063.910194063923


### 3.3 Imputación (strategy='most_frequent')

In [75]:
# 3.3.1 Imputation (media)
my_imputer = SimpleImputer(strategy='most_frequent')

imputed_mf_X_train_extended = pd.DataFrame(my_imputer.fit_transform(X_train_extended))
imputed_mf_X_valid_extended = pd.DataFrame(my_imputer.transform(X_valid_extended))

# 3.3.2 Devolver el nombre dev las columnas, al imputar se elimanaron los nombres
imputed_mf_X_train_extended.columns = X_train_extended.columns
imputed_mf_X_valid_extended.columns = X_valid_extended.columns

# 3.3.3. Calculo del MAE para el Método
print("MAE from Approach 3 (An Extension to Imputation: most_frequent):")
print(score_dataset(imputed_mf_X_train_extended, imputed_mf_X_valid_extended, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation: most_frequent):
18012.925182648403


# Paso 4. Resumen de valoración a cada enfoque

In [76]:
print("MAE (Eliminar columnas con valores faltantes):")
print(score_dataset(reduce_X_train, reduce_X_valid, y_train, y_valid))
print("MAE (Imputación Simple: media):")
print(score_dataset(imputed_mean_X_train, imputed_mean_X_valid, y_train, y_valid))
print("MAE (Imputation Simple: mediana):")
print(score_dataset(imputed_median_X_train, imputed_median_X_valid, y_train, y_valid))
print("MAE (Imputation Simple: most_frequent):")
print(score_dataset(imputed_mf_X_train, imputed_mf_X_valid, y_train, y_valid))
print("MAE from Approach 3 (An Extension to Imputation: mean):")
print(score_dataset(imputed_mean_X_train_extended, imputed_mean_X_valid_extended, y_train, y_valid))
print("MAE from Approach 3 (An Extension to Imputation: median):")
print(score_dataset(imputed_median_X_train_extended, imputed_median_X_valid_extended, y_train, y_valid))
print("MAE from Approach 3 (An Extension to Imputation: most_frequent):")
print(score_dataset(imputed_mf_X_train_extended, imputed_mf_X_valid_extended, y_train, y_valid))

MAE (Eliminar columnas con valores faltantes):
17837.82570776256
MAE (Imputación Simple: media):
18062.894611872147
MAE (Imputation Simple: mediana):
17791.59899543379
MAE (Imputation Simple: most_frequent):
17956.065479452056
MAE from Approach 3 (An Extension to Imputation: mean):
18148.417180365297
MAE from Approach 3 (An Extension to Imputation: median):
18063.910194063923
MAE from Approach 3 (An Extension to Imputation: most_frequent):
18012.925182648403


* En resumen, la **imputación simple** utilizando la mediana, resultó más óptima.

# Paso 5. Generar predicciones de prueba. X_test
* En este paso final, se utilizará el enfoque más óptimo que resultó de abordar los valores faltantes.
* Una vez que haya preprocesado las funciones de entrenamiento y validación, se entrenará y evaluará un modelo de RandomForest.
* Luego, se procesará previamente los datos de la prueba antes de generar predicciones.

## Parte A. Preprocesar los datos de entrenamiento y validación.

* Se establecen los DataFrames preprocesados en **final_X_train** y **final_X_valid**.
* Para que este paso se debe asegurar:
  1. los DataFrames preprocesados tienen el mismo número de columnas,
  2. los DataFrames preprocesados no tienen valores faltantes,
  3. **final_X_train** e **y_train** tienen el mismo número de filas, y
  4. **final_X_valid** e **y_valid** tienen el mismo número de filas.
  5. Definir y ajustar las variables finales en el modelo entrenado
  6. Obtener predicciones de validación y MAE

In [78]:
# 1. Funciones de validación y entrenamiento preprocesadas.
# Los fresultados obtenidas por el enfoque imputación simple (mediana) fueron mejores.
# Los elegimos para nuestro modelo final y se definen las variables finales.
final_X_train = imputed_median_X_train
final_X_valid = imputed_median_X_valid

In [93]:
# 2.1 Revisar las mismas columnas en los dataframe de entrenamiento y validación
# Función para comparar columnas en dos dataframe

def comparar_columnas(df_a, df_b, nombre_df_a, nombre_df_b, considerar_orden=False):
    """
    Compara si dos DataFrames tienen los mismos nombres de columnas.

    Parámetros:
    df_a (pd.DataFrame): Primer DataFrame.
    df_b (pd.DataFrame): Segundo DataFrame.
    nombre_df_a (str): Nombre del primer DataFrame para imprimir en el mensaje.
    nombre_df_b (str): Nombre del segundo DataFrame para imprimir en el mensaje.
    considerar_orden (bool): Si se debe considerar el orden de las columnas.

    Retorna:
    bool: True si los DataFrames tienen las mismas columnas, False en caso contrario.
    """
    if considerar_orden:
        # Verificar si el orden de las columnas es el mismo
        mismas_columnas = list(df_a.columns) == list(df_b.columns)
    else:
        # Verificar si los conjuntos de columnas son iguales, sin considerar el orden
        mismas_columnas = set(df_a.columns) == set(df_b.columns)

    if mismas_columnas:
        print(f"Los DataFrames '{nombre_df_a}' y '{nombre_df_b}' tienen las mismas columnas iguales a {df_a.shape[1]}.")
    else:
        columnas_df_a = set(df_a.columns)
        columnas_df_b = set(df_b.columns)
        columnas_faltantes_a = columnas_df_b - columnas_df_a
        columnas_faltantes_b = columnas_df_a - columnas_df_b

        print(f"Los DataFrames '{nombre_df_a}' y '{nombre_df_b}' NO tienen las mismas columnas.")
        if columnas_faltantes_a:
            print(f"Columnas en '{nombre_df_b}' que faltan en '{nombre_df_a}': {columnas_faltantes_a}")
        if columnas_faltantes_b:
            print(f"Columnas en '{nombre_df_a}' que faltan en '{nombre_df_b}': {columnas_faltantes_b}")


In [94]:
comparar_columnas(final_X_train, final_X_valid, "final_X_train", "final_X_valid")

Los DataFrames 'final_X_train' y 'final_X_valid' tienen las mismas columnas iguales a 36.


In [99]:
# 2.2 Revisar que no existan valores faltantes en los dataframe
# Función para verificar valores fantanes en un dataframe
def verificar_valores_faltantes(df, nombre_df ):
    # Verificar si hay valores faltantes en el DataFrame
    faltantes = df.isnull().values.any()

    # Imprimir resultado
    if not faltantes:
        print(f"El DataFrame '{nombre_df}' no tiene valores faltantes.")
    else:
        print(f"El DataFrame '{nombre_df}' tiene valores faltantes.")


In [100]:
verificar_valores_faltantes(final_X_train, "final_X_train")
verificar_valores_faltantes(final_X_valid, "final_X_valid")

El DataFrame 'final_X_train' no tiene valores faltantes.
El DataFrame 'final_X_valid' no tiene valores faltantes.


In [114]:
# 3.Revisar que final_X_train tiene el mismo número de filas que y_train
print(f"final_X_train tiene {final_X_train.shape[0]} filas")
print(f"y_train tiene {y_train.shape[0]} filas")

final_X_train tiene 1168 filas
y_train tiene 1168 filas


In [115]:
# 4. Revisar que final_X_valid tiene el mismo número de filas que y_valid
print(f"final_X_valid tiene {final_X_valid.shape[0]} filas")
print(f"y_valid tiene {y_valid.shape[0]} filas")

final_X_valid tiene 292 filas
y_valid tiene 292 filas


In [104]:
# 5. Definir y ajustar las variables finales en el modelo entrenado
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

RandomForestRegressor(random_state=0)

In [105]:
# 6. Obtener predicciones de validación y MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
17791.59899543379


## Parte B. Preprocesar los datos de prueba (test)
* Asegurarse de utilizar un método que coincida con la forma en que se procesó previamente los datos de entrenamiento y validación, y configurar las funciones de prueba preprocesadas en **final_X_test**.
* Luego, utilizar las funciones de prueba preprocesadas y el modelo entrenado para generar predicciones de prueba en **preds_test**.
* Asegurarse de:
  1. el DataFrame de prueba preprocesado **no tiene valores faltantes**, y
  2. **final_X_test** tiene el mismo número de filas que **X_test**.

In [112]:
# Tamaño de X_test
print(f"X_test, tiene {X_test.shape[0]} filas y {X_test.shape[1]} columnas")

X_test, tiene 1459 filas y 36 columnas


* Los datos de final_X_test se deben preprocesar con el mismo método de preprocesamiento que los datos de entrenamiento y validación.(final_X_train y final_X_valid)
* Los **datos de entrenamiento y validación** se obtuvieron con el método imputación simple (mediana)

**Importante**

* Como los nombres de las caracteristicas (variables predictoras) deben coincidir con los que se aprobaron durante el ajuste del modelo (fit), se debe tener cuidado en mantener tambien esas columnas en la data test (X_test).

In [117]:
# 1. Preprocesar X_test tambien con el metodo de imputación simple (mediana)

from sklearn.impute import SimpleImputer

mi_imputacion_final = SimpleImputer(strategy='median')
imputed_X_test = pd.DataFrame(mi_imputacion_final.fit_transform(X_test))

# 2. Devolver nombre a columnas eliminadas por la imputación.

imputed_X_test.columns = X_test.columns

# 3. Preprocesar Data de prueba con el mismo método ya imputado
final_X_test = imputed_X_test

In [118]:
# 4. Verificar que final_X_test no tiene valores faltantes
verificar_valores_faltantes(final_X_test, "final_X_test")

El DataFrame 'final_X_test' no tiene valores faltantes.


In [120]:
# 5. Revisar que final_X_test tiene el mismo número de filas que X_test
print(f"final_X_test tiene {final_X_test.shape[0]} filas")
print(f"X_test tiene {X_test.shape[0]} filas")

final_X_test tiene 1459 filas
X_test tiene 1459 filas


In [121]:
# 6.Obtener la prediciòn de prueba
preds_test = model.predict(final_X_test)

In [123]:
preds_test

array([125985.5 , 154599.5 , 180070.24, ..., 154751.44, 107387.  ,
       229281.55])

In [124]:
# 7. Guardar las predicciones en un archivo
output = pd.DataFrame({'Id': X_test.index,
                      'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)